In [1]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd

In [5]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[300:] 

In [ ]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoCropper(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    
    data: DataFrame
    Mouse moving location, by pixel
    
    !!! We are assuming that the mouse at least reaches the boundary once
    '''
    x = sorted(data['x'], reverse = True)
    y = sorted(data['y'], reverse = True)
    
    # Try to exclude outliers
    for i in range(10):
        xMax = round(x[i + 1]) if x[i] - x[i + 1] >= 1 else round(x[i])
        
        xMin = round(x[-i-1]) if x[-i] - x[-i-1] >= 1 else round(x[-i])
            
        yMax = round(y[i + 1]) if y[i] - y[i + 1] >= 1 else round(y[i])

        yMin = round(y[-i-1]) if y[-i] - y[-i-1] >= 1 else round(y[-i])
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
    
    return xMin, yMin, lengthX, lengthY



def dataModifier(data):
    '''
    Task: Modify the location data based on the cropped video
    
    data: DataFrame
    '''
    
    x, y, lengthX, lengthY = getCoord(data)
    filtered = data.copy(deep = True)
    filtered = filtered[(filtered['y']>x) & (filtered['x']>y)]
    filtered['x'] = filtered['x'] - y
    filtered['y'] = filtered['y'] - x
    filtered['coords'] = list(range(1,len(filtered)+1))
    filtered.index = list(range(1,len(filtered)+1))
    
    return filtered

In [ ]:
dataModifier(move_data)

In [ ]:
def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    return (direction, degreeL)
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.


In [ ]:
degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))

In [ ]:
degrees(atan2(10,-20))

In [ ]:
degrees(atan2(30,50))

In [ ]:
x = (1,1)
p = (1,-1)
z = (-1,-1)
y = (-1,1)

In [ ]:
degrees(2.158798930342464)

In [3]:
from math import radians, degrees, atan2
 
def quadrant(x, y): 
    if ((x[0] > 0 and y[0] > 0) or (x[0] < 0 and y[0] < 0)) and ((x[1] > 0 and y[1] > 0) or (x[1] < 0 and y[1] < 0)):
            return True
    else:
        return False
    
def getAngle(a, b, c):
    cb = atan2(c[1]-b[1], c[0]-b[0])
    ab = atan2(a[1]-b[1], a[0]-b[0])

    # If they are in the same quadrant: acute angle
    if quadrant(c, b) and quadrant(a, b):
        angle = degrees(cb - ab)
    else:
    # Obtuse angle or in different quadrant
        rad360 = radians(360)
        cb = rad360 + cb if cb < 0 else cb
        ab = rad360 + ab if ab < 0 else ab
        angle = abs(degrees(cb - ab))
    return angle + 360 if angle < 0 else angle


3.2625154121710547

In [85]:
def calDist_Angle(data, enclosure):
    '''
    Task: Calculate distance and angle between enclosure and defeated mouse
    
    Parameters
    ----------
    data: DataFrame
        Read location data
    
    enclosure: tuple
        enclosure's coordinate, ex. (68,204)
    '''
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    mid = (p1.x + p2.x) / 2, (p1.y + p2.y) / 2
    mid = np.array(mid)
    x = np.array(p1.x)
    y = np.array(p1.y)
    angle = []
    for i in range(len(p1)):
        # Set middle point of the mouse to be (0,0), then calculate the corresponding relative converted coordinates
        # Order: head relative location, central point, enclosure relative location
        angle.append(getAngle((x[i] - mid[0][i], y[i] - mid[1][i]),(0, 0), (enclosure[0] - mid[0][i], enclosure[1] - mid[1][i])))
    return angle

In [86]:
calDist_Angle(move_data, (68,204))

[22.69577137639606,
 29.645005823001444,
 32.53829415653658,
 34.20020791824113,
 41.18740958892423,
 52.756901372051296,
 62.44996845963485,
 68.95507658121079,
 72.52744558972842,
 77.1947929088088,
 79.7836011344901,
 90.52106815181155,
 87.69038161024442,
 79.72039806975035,
 87.26835039982333,
 90.37234926714852,
 89.36203032926271,
 86.2177673663642,
 94.41796049032385,
 96.20522150292844,
 90.50574438858669,
 96.25492814399114,
 101.01640498827638,
 103.79019994570147,
 107.58820322610707,
 113.67839760359261,
 108.61187781022473,
 108.27732576600638,
 109.65946645470501,
 112.26423622794462,
 118.57990204676634,
 120.90791746804915,
 118.41104090973633,
 120.17912599668631,
 127.33326887909612,
 120.06432082374182,
 123.58578205115617,
 104.72853079978232,
 93.46006447060863,
 85.00770981560024,
 71.41709745869338,
 50.42309856033789,
 16.205932945387953,
 9.956103733258233,
 28.295142941968948,
 29.481299950802796,
 16.873534126896065,
 13.695753096369472,
 14.919100013216646,

In [ ]:
csvfile

In [ ]:
test = move_data
test

In [ ]:
import numpy as np
pd.options.mode.chained_assignment = None
p['coords'] = list(range(1,len(p)+1))
p.index = list(range(1,len(p)+1))

In [ ]:
p

In [ ]:
p = test[(test['y']>54) & (test['x']>38)]

In [ ]:
test = move_data['x'] - 38
test[test>0]

In [ ]:
def videoWriter(task, filename, outputName, corrX = None, corrY = None, dist = 320, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (dist, dist)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out

def fieldOfView(data, filename, outputName, radius, rotationAngle, color, viewRange, thickness, displayVideo = False):
    '''    
    Task: draw sector for field of view

    All input parameters must be Int

    PARAMETERS:

    data: mouse movement that read from .h5 file, including head and tail. 

    angle: 0, rotation angle

    radius: radius of the sector, millimeter

    color: sector line color

    viewRange: range that the mouse has vision on

    thickness: the thickness of the sector line

    '''

    #Obtain coordinates data for head and tail, as well as the head direction by degrees
    

    head = zip(round(data['x']).astype(int), round(data['y']).astype(int))
    tail = zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int))
    degrees = head_dir(data)[1]
    
    #Get pixel to mm conversion
    x, y, lengthX, lengthY = getCoord(data)
    length = lengthX if lengthX < lengthY else lengthY
    convert = 440/length # video side length 440mm, square
    radius = round(radius / convert)
    
    
    #cap: Read video
    #out: Write video
    video = videoWriter("field of view", filename, outputName, displayVideo)
    cap = video[0]
    out = video[1]

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    while(cap.isOpened()):
        try:
            for center, angle in tqdm(zip(head, degrees)):
                ret,img = cap.read()
            #draw the sector representing the field of view of mouse
                angle = int(round(angle))
                startAngle = angle - viewRange
                endAngle = angle + viewRange
                img2 = drawSector(img, radius, center, rotationAngle, startAngle, endAngle, color, thickness)
                out.write(img2)
                    
                if displayVideo == True:
                    cv2.imshow('Draw field of view video',img2) 
                    
                # Break the loop when the last frame has been rotated
                if count == frameCnt:
                    print (filename, 'successfully ', "added field of view!!!")
                    return
                else:
                    count += 1

                k=cv2.waitKey(30) & 0xff
                #once you inter Esc capturing will stop
                if k==27:
                    break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [ ]:
BLACK = (0,0,0)
from math import degrees, atan2
def drawSector(image, radius, center, angle, startAngle, endAngle, color, thickness):
    # Ellipse parameters
    axes = (radius, radius)

    cv2.ellipse(image, center, axes, angle, startAngle, endAngle, color, thickness)
    points = cv2.ellipse2Poly(center, axes, angle, startAngle, endAngle, thickness)
    point1 = tuple(points[0]) # End's coordinates
    point2 = tuple(points[-1]) # Another end's coordinates
    cv2.line(image, center, point1, color)
    cv2.line(image, center, point2, color)
    return image

def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    if __name__ == "__main__": 
        return (direction, degreeL)

In [ ]:
x = dataModifier(move_data)

In [ ]:
fieldOfView(x, '1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest1.mp4", 50, 0, BLACK, 60, 1)

In [ ]:
x, y, lengthX, lengthY = getCoord(move_data)

In [ ]:
x, y, lengthX, lengthY

In [ ]:
convertX = 440/lengthX
convertY = 440/lengthY
print(convertX, convertY)

In [ ]:
440/331

In [ ]:
videoCropper('1034 SI_B, Aug 15, 13 7 49 rotated.mp4', "cropTest rotated.mp4", y, x, lengthY, lengthX)

In [ ]:
##Question: choose length of the edge in order to be converted in video processing